In [ ]:
import numpy as np
import os

from cil.utilities.display import show2D, show_geometry
from cil.utilities.jupyter import islicer, link_islicer

from cil.io import TXRMDataReader

from cil.processors import TransmissionAbsorptionConverter

from cil.processors import CentreOfRotationCorrector
from cil.plugins.tigre import FBP

from cil.recon import FDK

from cil.processors import Slicer

from cil.optimisation.algorithms import FISTA
from cil.plugins.ccpi_regularisation.functions import FGP_TV
from cil.optimisation.functions import LeastSquares
from cil.plugins.tigre import ProjectionOperator

## Load the data

Load a ZEISS dataset and explore the data

In [ ]:
filename = "/mnt/materials/SIRF/Fully3D/CIL/Walnut/valnut_2014-03-21_643_28/tomo-A/valnut_tomo-A.txrm"

data = TXRMDataReader(file_name=filename).read()
show_geometry(data.geometry)
print (data.geometry)

In [ ]:
sl1 = islicer(data, size=9, minmax=(0, 1))

## Convert Transmission to Absorption

In [ ]:
data = TransmissionAbsorptionConverter()(data)

In [ ]:
data.reorder(order='tigre')
corr = CentreOfRotationCorrector.image_sharpness(FBP=FBP)
corr(data, out=data)

## Reconstruct with FDK

In [ ]:
%%time 
recon3d = FDK(data).run()

In [ ]:
islicer(recon3d, minmax=(-0.003, 0.05), size=9, cmap='hot')

## Reconstruction with fewer projections 

In [ ]:
data2d = data.get_slice(vertical='centre')
show_geometry(data2d.geometry)

In [ ]:
recon = recon3d.get_slice(vertical='centre')

Reduce the amount of projection to 10% by selecting a projection every 10.

In [ ]:
reduce_factor = 10

data_reduced = Slicer(
    roi={'angle': (0,-1,reduce_factor)}
    )(data2d)

Reconstruct the reduced dataset again with FDK.

In [ ]:
recon_reduced = FDK(data_reduced).run()

In [ ]:
show2D([recon, recon_reduced], fix_range=(-0.01, 0.06), cmap='hot', title=['full dataset', 'reduced dataset'])

## Iterative Reconstruction

$$\Large Au = b $$

* $A$ system matrix
* $b$ acquisition data
* $u$ reconstructed volume

$$  \Large u^* = \text{argmin} \left[ F(u, b) + \alpha G(u) \right] $$


* data fitting function, $ F = || Au - b ||^2_2 $ 
* regularisation parameter, $\alpha$ 
* $G = TV(u)$

In [ ]:
A = ProjectionOperator(ig,data_reduced.geometry)

In [ ]:
f = LeastSquares(A=A, b=data_reduced)

In [ ]:
alpha = 0.02

g = alpha * FGP_TV(device='gpu')

Create an algorithm to solve the optimisation problem. Here we choose the algorithm named FISTA.

In [ ]:
algo = FISTA(initial=ig.allocate(0), 
             f=f, 
             g=g, 
             max_iteration=100, 
             update_objective_interval=10)

Run the algorithm for 100 iterations.

In [ ]:
algo.run(100)

In [ ]:
show2D([recon_reduced, algo.solution], fix_range=(-0.01, 0.06),
      title=['FDK reduced dataset', 'TV reduced dataset'], cmap='hot')

In [ ]:
show2D(recon-algo.solution, cmap='seismic', fix_range=(-0.05, 0.05))